In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from collections import defaultdict
import glob
from collections import Counter
import os

import tqdm
import optuna
import warnings
warnings.filterwarnings('ignore')

# Сравнение коррелированных признаков по их важности

In [ ]:
corr_features = ['feature_454', 'feature_103', 'feature_495', 'feature_388', 'feature_224', 'feature_428', 'feature_381', 'feature_208', 'feature_285', 'feature_178', 'feature_379', 'feature_410', 'feature_247', 'feature_141', 'feature_173','feature_434']

In [21]:
categorial_features = ['feature_17', 'feature_21', 'feature_22', 'feature_27', 'feature_31', 'feature_43', 'feature_61', 'feature_64', 'feature_80', 'feature_83', 'feature_92', 'feature_133', 'feature_143', 'feature_166', 'feature_191', 'feature_201', 'feature_209', 'feature_251', 'feature_253', 'feature_299', 'feature_300', 'feature_343', 'feature_347', 'feature_382', 'feature_392', 'feature_405', 'feature_406', 'feature_423', 'feature_446', 'feature_449', 'feature_459', 'feature_490', 'feature_492']

['feature_14', 'feature_462', 'feature_44']

In [10]:
chosen_features_corr = ['feature_80', 'feature_421', 'feature_47', 'feature_424', 'feature_234', 'feature_48', 'feature_383', 'feature_168', 'feature_157', 'feature_490', 'feature_244', 'feature_239', 'feature_186', 'feature_386', 'feature_11', 'feature_495', 'feature_35', 'feature_332', 'feature_100', 'feature_97', 'feature_153', 'feature_119', 'feature_151', 'feature_147', 'feature_467', 'feature_390', 'feature_365', 'feature_439', 'feature_18', 'feature_178', 'feature_154', 'feature_334', 'feature_353', 'feature_138', 'feature_364', 'feature_114', 'feature_81', 'feature_435', 'feature_318', 'feature_341', 'feature_139', 'feature_323', 'feature_164', 'feature_296', 'feature_86', 'feature_68', 'feature_265', 'feature_428', 'feature_230', 'feature_336', 'feature_470', 'feature_45', 'feature_51', 'feature_264', 'feature_166', 'feature_12', 'feature_366', 'feature_356', 'feature_131', 'feature_14', 'feature_462', 'feature_44', 'feature_87', 'feature_452', 'feature_82', 'feature_481', 'feature_456', 'feature_129', 'feature_325', 'feature_340', 'feature_194', 'feature_132', 'feature_273', 'feature_378', 'feature_66', 'feature_345', 'feature_444', 'feature_305', 'feature_246', 'feature_369', 'feature_270', 'feature_242', 'feature_42', 'feature_493', 'feature_461', 'feature_197', 'feature_331', 'feature_349', 'feature_412', 'feature_458', 'feature_200', 'feature_373', 'feature_480', 'feature_494', 'feature_473', 'feature_380', 'feature_199', 'feature_272', 'feature_402', 'feature_498', 'feature_238', 'feature_351', 'feature_310', 'feature_76', 'feature_223', 'feature_455', 'feature_360', 'feature_359', 'feature_434', 'feature_314', 'feature_408', 'feature_361', 'feature_342', 'feature_394', 'feature_363', 'feature_311', 'feature_291', 'feature_407', 'feature_212', 'feature_137', 'feature_19', 'feature_350', 'feature_1', 'feature_295', 'feature_320', 'feature_451', 'feature_344', 'feature_276', 'feature_357', 'feature_286', 'feature_328', 'feature_368', 'feature_352', 'feature_279', 'feature_346', 'feature_472', 'feature_358', 'feature_303', 'feature_464', 'feature_278', 'feature_384', 'feature_222', 'feature_347', 'feature_398', 'feature_319', 'feature_292', 'feature_301', 'feature_294', 'feature_330', 'feature_333', 'feature_367', 'feature_309']

In [15]:
# Считываем все файлы один раз и сохраняем их в список
# train_dfs = [pd.read_csv(f"Files/Train/train_{i+1}.csv") for i in range(1, 6)]

# Список замен для фичей
feature_replacements = [
    ('feature_98', 'feature_178'),
    ('feature_98', 'feature_285'),
    ('feature_98', 'feature_381'),
    ('feature_239', 'feature_388'),
    ('feature_323', 'feature_325'),
    ('feature_323', 'feature_379'),
    ('feature_424', 'feature_454')
]

# Перебор замен и расчет roc_auc
full_roc_auc_results = []

for original_feature, replacement_feature in feature_replacements:
    current_features = [f if f != original_feature else replacement_feature for f in chosen_features_corr]
    
    # Перебор предзагруженных файлов для каждой замены фичей
    full_roc_auc = []
    for i, train_df in enumerate(train_dfs):
        col_for_drop = ['id', 'smpl']
        super_clean_df = train_df.drop(columns=col_for_drop).copy()
        
        super_clean_df[categorial_features] = super_clean_df[categorial_features].astype(str)
        new_train_df = super_clean_df.copy()

        # Формирование датасета с измененными фичами
        new_train_df_imp_columns = new_train_df[current_features + ['feature_173'] + ['target']]
        X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_split(new_train_df_imp_columns, 'target')
        
        model_categorial_f = list(np.intersect1d(categorial_features, new_train_df_imp_columns.columns))
        best_params = train_best_params[-1]
        
        # Инициализация и обучение CatBoost модели
        CatBoostModel = CatBoostClassifier(
            loss_function='CrossEntropy',
            random_state=42,
            task_type='GPU',
            verbose=False,
            **best_params
        )
        CatBoostModel.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            cat_features=model_categorial_f,
            plot=False,
            verbose=1000
        )
        
        # Предсказание и расчет ROC-AUC
        y_pred_proba = CatBoostModel.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        full_roc_auc.append(roc_auc)
    
    # Среднее значение roc_auc для текущей замены
    avg_roc_auc = np.mean(full_roc_auc)
    print(f"Среднее значение ROC-AUC при замене {original_feature} на {replacement_feature}: {avg_roc_auc}")
    full_roc_auc_results.append((original_feature, replacement_feature, avg_roc_auc))

0:	learn: 0.6733309	test: 0.6733179	best: 0.6733179 (0)	total: 43.5ms	remaining: 8.66s
199:	learn: 0.0816194	test: 0.0823408	best: 0.0823408 (199)	total: 5.62s	remaining: 0us
bestTest = 0.08234080107
bestIteration = 199
0:	learn: 0.6731503	test: 0.6731322	best: 0.6731322 (0)	total: 77.4ms	remaining: 15.4s
199:	learn: 0.0800061	test: 0.0796290	best: 0.0796290 (199)	total: 5.9s	remaining: 0us
bestTest = 0.0796290052
bestIteration = 199
0:	learn: 0.6743514	test: 0.6743571	best: 0.6743571 (0)	total: 55.3ms	remaining: 11s
199:	learn: 0.0949433	test: 0.0946663	best: 0.0946663 (199)	total: 5.47s	remaining: 0us
bestTest = 0.09466634423
bestIteration = 199
0:	learn: 0.6756934	test: 0.6756873	best: 0.6756873 (0)	total: 37.9ms	remaining: 7.54s
199:	learn: 0.1187514	test: 0.1190377	best: 0.1190377 (199)	total: 5.82s	remaining: 0us
bestTest = 0.1190377494
bestIteration = 199
0:	learn: 0.6782947	test: 0.6782743	best: 0.6782743 (0)	total: 28ms	remaining: 5.58s
199:	learn: 0.1639611	test: 0.1630830	be

**Вывод:**

feature_495 вместо feature_9\
'feature_50' на 'feature_428'